# Cross Decomposition Example

Original Model: 

$ \min \> x_1 + 2x_2 +3y + 4w\\
s.t. \> \>
x_1 + x_2 + y \geq 6\\
-3x_1 - 2x_2 + w \geq 7\\
10 \geq x,y,w \geq 0\\
$ 

Reformulated Model:

$\min \> 0.5x_{A1} + x_{A2}+ 0.5x_{B1} + x_{B2} + 3y + 4w\\
s.t. \> \>
x_{A1}+x_{A2}+ y \geq 6\\
-3x_{B1}+2x_{B2}+w\geq 7\\
x_{A1}-x_{B1} = 0\\
x_{A2}-x_{B2} = 0\\
$

Lagrange Master Problem:

$ \max \> \eta \\
s.t. \> \> \eta \leq \sum\kappa_s\\
\kappa \geq 0\\
$

Lagrange SP 1:

$ \min \> 0.5x_{A1} + x_{A2} + 3y +\lambda_1^k(x_{A1} + x_{A2})\\
   s.t. \> \> x_{A1}+x_{A2}+ y \geq 6\\ \\
   x_A,y \geq 0 \\
$

Lagrange SP 2:

$ \min \> 0.5x_{B1} + x_{B2} + 4w - \lambda_2^k(x_{B1} + x_{B2})\\
   s.t. \> \> -3x_{B1}+2x_{B2}+w\geq 7 \\
   x_B,w \geq 0 \\
$

Bender's Master Problem:

$\min \> \theta_1 + \theta_2 \\ 
s.t. \> \> x, \theta \geq 0\\
$


Bender's SP 1:

$ \min \> 0.5\bar x_1 + \bar x_2 + 3y \\
s.t. \> \>  - y \leq -6 + \bar x_1 + \bar x_2\\
y \geq 0 \\
$


Bender's SP 2:

$ \min \> 0.5\bar x_1 + \bar x_2 + 4w \\
s.t. \> \> - w \leq -7 -3\bar x_1 + 2\bar x_2 \\
w \geq 0 \\
$

Bender's DSP 1:

$ \max \> 0.5\bar x_1 + \bar x_2 + \mu_1(6 - \bar x_1 - \bar x_2)\\
s.t. \> \> \mu_1 \leq 3\\
\mu \geq 0 \\
$


Bender's DSP 2:

$ \max \> 0.5\bar x_1 + \bar x_2 + \mu_2(7 + 3\bar x_1 - 2\bar x_2)\\
s.t. \> \> \mu_2 \leq 4\\
\mu \geq 0 \\
$

Write BMP and BDSPs in Julia:

In [43]:
using JuMP
using Gurobi
BMP = Model(solver = GurobiSolver())
BDSP1 = Model(solver = GurobiSolver())
BDSP2 = Model(solver = GurobiSolver())

@variable(BMP, x[1:2] >= 0)
@variable(BMP, θ[1:2] >= 0)


@objective(BMP, Min, θ[1]+θ[2])

@variable(BDSP1, xbar[1:2])
@variable(BDSP1, μ>=0)

@constraint(BDSP1, μ<=3)
@objective(BDSP1, Max, 0.5*xbar[1] + xbar[2] + μ*(6-xbar[1]-xbar[2]))

@variable(BDSP2, xbar[1:2])
@variable(BDSP2, μ>=0)

@constraint(BDSP2, μ<=4)
@objective(BDSP2, Max, 0.5*xbar[1] + xbar[2] + μ*(7+3*xbar[1]-2*xbar[2]));

Write LMP and LSPs in Julia:

In [44]:
LMP = Model(solver = GurobiSolver())
LSP1 = Model(solver = GurobiSolver())
LSP2 = Model(solver = GurobiSolver())

@variable(LMP, κ[1:2])
@variable(LMP, η)
@variable(LMP, λ[1:2])

@constraint(LMP, η <= κ[1] + κ[2])

@objective(LMP, Max, η)

@variable(LSP1, xA[1:2]>=0)
@variable(LSP1, y>= 0)
@variable(LSP1, λ)

@constraint(LSP1, xA[1:2].<=10)
@constraint(LSP1, xA[1]+xA[2]+y>=6)
@objective(LSP1, Min, .5*xA[1]+xA[2]+3y+λ*(xA[1]+xA[2]))


@variable(LSP2, xB[1:2]>=0)
@variable(LSP2, w>= 0)
@variable(LSP2, λ)

@constraint(LSP2, xB[1:2].<=10)
@constraint(LSP2, -3xB[1]+2*xB[2]+w>=7)
@objective(LSP2, Min, .5*xB[1] + xB[2] + 4w - λ*(xB[1] + xB[2]));

Initialize $(\bar x_1, \bar x_2) = (0,0)$:

In [45]:
x = getindex(BDSP1, :xbar)
setlowerbound(x[1],0)
setupperbound(x[1],0)

setlowerbound(x[2],0)
setupperbound(x[2],0)

x = getindex(BDSP2, :xbar)
setlowerbound(x[1],0)
setupperbound(x[1],0)

setlowerbound(x[2],0)
setupperbound(x[2],0);

Initialize $(\lambda_1, \lambda_2) = (0,0)$:

In [46]:
λ = getindex(LSP1, :λ)
setupperbound(λ,0)
setlowerbound(λ,0)

λ = getindex(LSP2, :λ)
setupperbound(λ,0)
setlowerbound(λ,0);

# BDSPs to BMP

First we solve the BDSPs and make a cut to the BMP such that:

$\theta_s \geq \tau_sc^Tx + (A_1x-b_1)^Tu_s^k$

In [55]:
solve(BDSP1)
μ = getindex(BDSP1, :μ)

θ = getindex(BMP, :θ)
x = getindex(BMP, :x)
@constraint(BMP, θ[1]>= 0.5*x[1] + x[2] + getvalue(μ)*(6-x[1]-x[2]))

solve(BDSP2)
μ = getindex(BDSP2, :μ)

@constraint(BMP,  θ[2]>= 0.5*x[1] + x[2] + getvalue(μ)*(7+3x[1]-2*x[2]))

solve(BMP)
xbar = getindex(BDSP1, :xbar)
setlowerbound(xbar[1],getvalue(x[1]))
setupperbound(xbar[1],getvalue(x[1]))

setlowerbound(xbar[2],getvalue(x[2]))
setupperbound(xbar[2],getvalue(x[2]))

xbar = getindex(BDSP2, :xbar)
setlowerbound(xbar[1],getvalue(x[1]))
setupperbound(xbar[1],getvalue(x[1]))

setlowerbound(xbar[2],getvalue(x[2]))
setupperbound(xbar[2],getvalue(x[2]));

Optimize a model with 1 rows, 3 columns and 1 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 6e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [9e+00, 9e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 9.00000000e+00
Optimize a model with 1 rows, 3 columns and 1 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 7e+00]
  QObjective range [4e+00, 6e+00]
  Bounds range     [9e+00, 9e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 9.00000000e+00
Optimize a model with 6 rows, 4 columns and 14 nonzero

In [56]:
print(BMP)

Min θ[1] + θ[2]
Subject to
 θ[1] + 2.5 x[1] + 2 x[2] >= 18
 θ[2] - 12.5 x[1] + 7 x[2] >= 28
 θ[1] >= 3.0000000000028884
 θ[2] >= 3.500000000000036
 θ[1] - 0.5 x[1] - x[2] >= 0
 θ[2] - 0.5 x[1] - x[2] >= 0
 x[i] >= 0 for all i in {1,2}
 θ[i] >= 0 for all i in {1,2}


# BDSPs to LMP

We can also use the solutions from the BDSPs to make a cut to the LMP such that:

$\kappa_s \leq z^k_s + \lambda_sH_s\hat x_s^k$

In [57]:
zk1 = getobjectivevalue(BDSP1)
zk2 = getobjectivevalue(BDSP2)

xhat = getindex(BDSP1, :xbar)
κ = getindex(LMP, :κ)
λ = getindex(LMP, :λ)

xhat1 = getvalue(xhat[1])
xhat2 = getvalue(xhat[2])

@constraint(LMP, κ[1] <= zk1 + λ[1]*(xhat1+xhat2))

xhat = getindex(BDSP2, :xbar)
xhat1 = getvalue(xhat[1])
xhat2 = getvalue(xhat[2])

@constraint(LMP, κ[2] <= zk2 - λ[2]*(xhat1+xhat2))
print(LMP)

Max η
Subject to
 η - κ[1] - κ[2] <= 0
 κ[1] <= 18
 κ[2] <= 28
 κ[1] - 6.000000000002414 λ[1] <= 3.0000000000028884
 κ[2] + 3.499999999999995 λ[2] <= 3.500000000000036
 κ[1] - 9 λ[1] <= 9
 κ[2] + 9 λ[2] <= 9
 κ[i] for all i in {1,2}
 λ[i] for all i in {1,2}
 η


# LSPs to LMP

Now we move on and solve the LSPs. The solutions are then used to make a cut to the LMP such that:

$\kappa_s \leq \tau_sc^T\tilde x_s^k + \tau_s d^T\tilde y^k_s + \lambda_sH_s\tilde x_s^k$

In [58]:
solve(LSP1)
x = getindex(LSP1, :xA)
y = getindex(LSP1, :y)

zk1 = getobjectivevalue(LSP1)
xA1 = getvalue(x[1])
xA2 = getvalue(x[2])
y1 = getvalue(y)


solve(LSP2)
x = getindex(LSP2, :xB)
w = getindex(LSP2, :w)
zk2 = getobjectivevalue(LSP2)
xB1 = getvalue(x[1])
xB2 = getvalue(x[2])
w2 = getvalue(w)

println(xB1)
println(xB2)
println(w2)

κ = getindex(LMP, :κ)
λ = getindex(LMP, :λ)

@constraint(LMP, κ[1] <= 0.5*xA1 + xA2 + 3*y1 + λ[1]*(xA1 + xA2))
@constraint(LMP, κ[2] <= 0.5*xB1 + xB2 + 4*w2 - λ[2]*(xB1 + xB2))

print(getvalue(getindex(LSP1, :xA)))
print(LMP)

Optimize a model with 3 rows, 4 columns and 5 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [6e+00, 1e+01]
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.14786506e+01 -2.44413188e+01  0.00e+00 0.00e+00  2.17e+01     0s
   1   2.31507926e+01  8.22251658e+00  0.00e+00 0.00e+00  2.49e+00     0s
   2   1.86429651e+01  1.79148673e+01  0.00e+00 2.84e-16  1.21e-01     0s
   3   1.80008814e+01  1.79993978e+01  0.00e+00 0.00e+00  2.47e-04     0s
   4   1.80000009e+01  1.

# LMP to LSPs

In [59]:
print(LMP)
solve(LMP)
λ = getindex(LMP, :λ)

λ1 = getvalue(λ[1])
λ2 = getvalue(λ[2])

λ = getindex(LSP1, :λ)
setupperbound(λ,λ1)
setlowerbound(λ,λ1)

λ = getindex(LSP2, :λ)
setupperbound(λ,λ2)
setlowerbound(λ,λ2);

Max η
Subject to
 η - κ[1] - κ[2] <= 0
 κ[1] <= 18
 κ[2] <= 28
 κ[1] - 6.000000000002414 λ[1] <= 3.0000000000028884
 κ[2] + 3.499999999999995 λ[2] <= 3.500000000000036
 κ[1] - 9 λ[1] <= 9
 κ[2] + 9 λ[2] <= 9
 κ[1] - 3.088298763152688 λ[1] <= 10.279253092119376
 κ[2] + 2.6035595446891526 λ[2] <= 9.775083187191465
 κ[i] for all i in {1,2}
 λ[i] for all i in {1,2}
 η
Optimize a model with 9 rows, 5 columns and 17 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.600000000e+01


# LSPs to BMP

In [60]:
θ = getindex(BMP, :θ)
x = getindex(BMP, :x)
λ1 = getindex(LSP1, :λ)
λ2 = getindex(LSP2, :λ)

λ1 = getvalue(λ1)
λ2 = getvalue(λ2)

println(λ1)
println(λ2)

@constraint(BMP, θ[1]>=zk1-λ1*(x[1]+x[2]))
@constraint(BMP, θ[2]>=zk2+λ2*(x[1]+x[2]))

2.4999999999985127
-7.0


θ[2] + 7 x[1] + 7 x[2] >= 28.000000000015532

# BMP

In [61]:
solve(BMP)
print(getvalue(getindex(BMP, :x)))

Optimize a model with 8 rows, 4 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.100000000e+01
[1.0, 5.0]

Solution after two iterations: [1,5]

In [62]:
print(BMP)
print(LMP)

Min θ[1] + θ[2]
Subject to
 θ[1] + 2.5 x[1] + 2 x[2] >= 18
 θ[2] - 12.5 x[1] + 7 x[2] >= 28
 θ[1] >= 3.0000000000028884
 θ[2] >= 3.500000000000036
 θ[1] - 0.5 x[1] - x[2] >= 0
 θ[2] - 0.5 x[1] - x[2] >= 0
 θ[1] + 2.4999999999985127 x[1] + 2.4999999999985127 x[2] >= 17.999999999996504
 θ[2] + 7 x[1] + 7 x[2] >= 28.000000000015532
 x[i] >= 0 for all i in {1,2}
 θ[i] >= 0 for all i in {1,2}
Max η
Subject to
 η - κ[1] - κ[2] <= 0
 κ[1] <= 18
 κ[2] <= 28
 κ[1] - 6.000000000002414 λ[1] <= 3.0000000000028884
 κ[2] + 3.499999999999995 λ[2] <= 3.500000000000036
 κ[1] - 9 λ[1] <= 9
 κ[2] + 9 λ[2] <= 9
 κ[1] - 3.088298763152688 λ[1] <= 10.279253092119376
 κ[2] + 2.6035595446891526 λ[2] <= 9.775083187191465
 κ[i] for all i in {1,2}
 λ[i] for all i in {1,2}
 η
